In [2]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
df = pd.read_csv("./kddcup99-dataset/kddcup99.csv")

# mostra tamanho da base de dados
df.shape

(494020, 42)

In [4]:
# mostra todas as colunas
df.columns

Index(['duration', 'protocol_type', 'service', 'flag', 'src_bytes',
       'dst_bytes', 'land', 'wrong_fragment', 'urgent', 'hot',
       'num_failed_logins', 'logged_in', 'lnum_compromised', 'lroot_shell',
       'lsu_attempted', 'lnum_root', 'lnum_file_creations', 'lnum_shells',
       'lnum_access_files', 'lnum_outbound_cmds', 'is_host_login',
       'is_guest_login', 'count', 'srv_count', 'serror_rate',
       'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate',
       'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count',
       'dst_host_srv_count', 'dst_host_same_srv_rate',
       'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate',
       'dst_host_srv_diff_host_rate', 'dst_host_serror_rate',
       'dst_host_srv_serror_rate', 'dst_host_rerror_rate',
       'dst_host_srv_rerror_rate', 'label'],
      dtype='object')

In [5]:
# mostra as informações das colunas
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 494020 entries, 0 to 494019
Data columns (total 42 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   duration                     494020 non-null  int64  
 1   protocol_type                494020 non-null  object 
 2   service                      494020 non-null  object 
 3   flag                         494020 non-null  object 
 4   src_bytes                    494020 non-null  int64  
 5   dst_bytes                    494020 non-null  int64  
 6   land                         494020 non-null  int64  
 7   wrong_fragment               494020 non-null  int64  
 8   urgent                       494020 non-null  int64  
 9   hot                          494020 non-null  int64  
 10  num_failed_logins            494020 non-null  int64  
 11  logged_in                    494020 non-null  int64  
 12  lnum_compromised             494020 non-null  int64  
 13 

In [6]:
# lista todas colunas menos a primeira e a penúltima
input_cols = list(df.columns)[1:-1]

# define a coluna label como alvo
target_col = 'label'

# seleciona as colunas com valores numéricos 
numeric_cols = df.select_dtypes(include=np.number).columns.tolist()[:-1]
numeric_cols

['duration',
 'src_bytes',
 'dst_bytes',
 'land',
 'wrong_fragment',
 'urgent',
 'hot',
 'num_failed_logins',
 'logged_in',
 'lnum_compromised',
 'lroot_shell',
 'lsu_attempted',
 'lnum_root',
 'lnum_file_creations',
 'lnum_shells',
 'lnum_access_files',
 'lnum_outbound_cmds',
 'is_host_login',
 'is_guest_login',
 'count',
 'srv_count',
 'serror_rate',
 'srv_serror_rate',
 'rerror_rate',
 'srv_rerror_rate',
 'same_srv_rate',
 'diff_srv_rate',
 'srv_diff_host_rate',
 'dst_host_count',
 'dst_host_srv_count',
 'dst_host_same_srv_rate',
 'dst_host_diff_srv_rate',
 'dst_host_same_src_port_rate',
 'dst_host_srv_diff_host_rate',
 'dst_host_serror_rate',
 'dst_host_srv_serror_rate',
 'dst_host_rerror_rate']

In [7]:
from sklearn.preprocessing import MinMaxScaler
# MinMaxScaler transforma os valores em uma escala de 0 a 1 
# para poder interar com mais facilidade colunas de diferentes medidas 
scaler = MinMaxScaler()

# Calcula o mínimo e o máximo para ser usado pra escalar os valores
scaler.fit(df[numeric_cols])

# escala os valores do parâmetro utilizado
df[numeric_cols] = scaler.transform(df[numeric_cols])

In [8]:
from sklearn.preprocessing import LabelEncoder
# LabelEncoder funciona muito parecido com o enumerate, dando valores de 0 a n para as diferentes strings 
le = LabelEncoder()

# coloca a coluna label como alvo
target = df['label']

# realiza fit (gera os valores) e e retorna o transform (verifica as classes dos valores) na target
df['label'] = le.fit_transform(target)

# o mesmo do anterior com o protocol_type
df['protocol_type'] = le.fit_transform(df['protocol_type'])

# o mesmo do anterior com o service
df['service'] = le.fit_transform(df['service'])

# o mesmo do anterior com a flag
df['flag'] = le.fit_transform(df['flag'])

In [9]:
from sklearn.model_selection import train_test_split
# serpara arrays ou matrizes em subsets randomicos de teste e treinamento
train_df, test_df = train_test_split(df, test_size=0.3, random_state=42)

print(train_df.shape)
print(test_df.shape)

(345814, 42)
(148206, 42)


In [10]:
# apenas copia valores para novas variaveis
train_inputs = train_df[input_cols].copy()
train_targets = train_df[target_col].copy()
test_inputs = test_df[input_cols].copy()
test_targets = test_df[target_col].copy()

In [11]:
# classificador de floresta randomico
# ver melhor a definição aqui: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html
from sklearn.ensemble import RandomForestClassifier
# transformador para selecionar características baseadas em pesos de importância
from sklearn.feature_selection import SelectFromModel

# utiliza o classificador de floresta com estimador = 5 e estado randomico de 42
sel = SelectFromModel(RandomForestClassifier(n_estimators = 5, random_state=42))

# da fit no meta-transformador SelectFromModel
sel.fit(train_inputs, train_targets)

# recebe as colunas que foram selecionadas
selected_feat = train_inputs.columns[(sel.get_support())]

print(selected_feat)
print(len(selected_feat))

Index(['protocol_type', 'service', 'srv_count', 'diff_srv_rate',
       'dst_host_same_srv_rate', 'dst_host_same_src_port_rate'],
      dtype='object')
6


In [14]:
# classificador de floresta randomico
from sklearn.ensemble import RandomForestClassifier

# utiliza o classificador de floresta com estimador = 1000 e estado randomico de 42
rf = RandomForestClassifier(n_estimators = 2000, random_state = 42)
# realiza o fit passando os inputs que irão receber o treinamento, com as colunas selecionadas no passo anterior e os alvos
rf.fit(train_inputs[selected_feat], train_targets);

# realiza a predição das classes com os inputs de test com as colunas selecionadas pelo feat
preds_rf = rf.predict(test_inputs[selected_feat])

# classificador de acuracia
from sklearn.metrics import accuracy_score

# com base nas classes observadas anteriormente, é verificado quais delas se comportam com o esperado
score_rf = accuracy_score(test_targets, preds_rf)
score_rf

# 0.9936642241204809

0.9936507293901732

In [21]:
# classificador de decisão de floresta
# faz basicamente a mesma coisa que o anterior, a diferença é que agora não utiliza mais os estimadores, porém o resto é igual
from sklearn.tree import DecisionTreeClassifier
dc = DecisionTreeClassifier()
dc.fit(train_inputs[selected_feat], train_targets);
preds_dc = dc.predict(test_inputs[selected_feat])
score_dc = accuracy_score(test_targets, preds_dc)

score_dc

0.9934685505310177

In [22]:
# classificador de k vizinhos
# também semelhante aos dois últimos, porém utiliza a parte de k vizinhos para verificar a acuracia
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=7)
knn.fit(train_inputs[selected_feat], train_targets)
preds_knn = knn.predict(test_inputs[selected_feat])
score_knn = accuracy_score(test_targets, preds_knn)

score_knn

0.9920448564835431